In [15]:
import requests
import polars as pl
from functools import partial
from pathlib import Path
from IPython.display import Markdown

from thematic_analysis.models import ThemeRequest, SurveyAnswer
from thematic_analysis.types import ThemeStrategy
from thematic_analysis.strategies.prompt_strategy import analyze_themes_with_prompt

In [2]:
df = pl.read_csv(Path("../data/wallmart.csv"))
df.head()

Session ID,Date started,"Imagine you are writing a complaint letter to the head of Walmart...in detail, tell us the things you would complain about and why they matter?",What do you think Target does better than Walmart?,What do you LOVE most about Walmart?,Gender,Age group,State,Who does it better? Target or Walmart?
str,str,str,str,str,str,str,str,str
"""94f2d4c3-b513-411c-b505-a11290…","""10/02/2022 01:03""","""The customer service that your…","""Their overall approach to cust…","""There's nothing I LOVE about W…","""Male""","""45-54""","""Texas""","""Target"""
"""1797c6f2-c501-44b7-b549-a33c29…","""10/02/2022 01:03""","""I would complain about how mor…",null,"""I love the diversity of produc…","""Female""","""18-24""","""Texas""","""Walmart"""
"""20e1a746-8311-450b-ba54-62d15f…","""10/02/2022 01:03""","""Why do you quit taking cash at…","""Target still has a snack bar u…","""The employees. They try when …","""Male""","""55-64""","""Texas""","""Target"""
"""234eb679-ffb8-4e5e-a45a-e50a7c…","""10/02/2022 01:03""","""You don't just go into Walmart…",null,"""In the small town that I live …","""Male""","""35-44""","""Texas""","""Walmart"""
"""511293b3-42e1-4712-a3a9-8838e0…","""10/02/2022 01:03""","""East Waco needs a Walmart neig…",null,"""The low price""","""Male""","""25-34""","""Texas""","""Walmart"""


In [3]:
def create_survey_answer_list(
    df: pl.DataFrame, answer_column: str, id_column: str = "Session ID"
) -> list[SurveyAnswer]:
    """
    Quick helper function for testing different endpoints
    """
    return [
        SurveyAnswer(id=id, answer=answer)
        for id, answer in zip(df[id_column], df[answer_column])
    ]

In [26]:
async def display_result(request: ThemeRequest) -> None:
    response = await analyze_themes_with_prompt(request)
    display(Markdown(f"## {request.question}"))
    for theme in response.themes:
        display(Markdown(theme.markdown)) 

In [23]:
question_columns = df.columns[2:5]

partial_themes = [
    partial(
        ThemeRequest,
        question=question,
        answers=create_survey_answer_list(df, question)
    )
    for question in question_columns
]

# Prompt Strategy Results

In [27]:
for theme in partial_themes:
    request = theme(strategy=ThemeStrategy.PROMPT)
    await display_result(request)

## Imagine you are writing a complaint letter to the head of Walmart...in detail, tell us the things you would complain about and why they matter?

**Poor Customer Service**

Numerous customer feedback entries highlight dissatisfaction with Walmart's customer service. Many respondents noted rude or dismissive attitudes from staff, indicating a lack of helpfulness or friendliness. This negative interaction often left customers feeling as if their needs were not valued or prioritized. The prevailing sentiment is that an improvement in employee training and a focus on customer engagement could greatly enhance the shopping experience.

**Inadequate Checkout Options**

A significant number of comments mention the prevailing issue of long checkout lines and a scarcity of open registers. Customers express frustration that, despite many self-checkout stations, there are often few staffed registers operating, leading to prolonged waits. This situation is exacerbated during peak shopping times, suggesting a need for better staffing strategies to improve efficiency and reduce wait times.

**Store Cleanliness and Organization**

Customers frequently mentioned concerns about the cleanliness and organization of Walmart stores. Reports of messy aisles, unkempt restrooms, and dirty surfaces were common, which detracts from the overall shopping experience. Shoppers are particularly sensitive to hygiene, especially in light of the ongoing pandemic, indicating that increased maintenance and better organizational practices could significantly enhance customer satisfaction.

**Stock Availability Issues**

Many customers voiced complaints regarding the inconsistency in product availability, with several noting that items they frequently purchase are often out of stock. This unpredictability in inventory can be particularly frustrating for dedicated customers who rely on Walmart for essential goods. Addressing stock management practices could lead to improved customer retention and satisfaction.

...

## What do you LOVE most about Walmart?

**Affordability and Low Prices**

A prominent theme across customer feedback is the appreciation for Walmart's low prices. Many customers emphasize the affordability of items, stating that it allows them to save money on everyday essentials. This sentiment highlights the significance of competitive pricing in attracting and retaining customers, making Walmart a preferred choice for budget-conscious shoppers. The emphasis on low prices indicates that maintaining this aspect is crucial for customer satisfaction and loyalty.

**Variety of Products**

Customers frequently mention Walmart's wide selection of products as a major benefit. They appreciate the store's ability to provide almost anything they need, from groceries to electronics, all in one place. This theme underscores the convenience Walmart offers, as it allows shoppers to fulfill diverse needs without having to visit multiple stores. The broad variety caters to many demographics and increases shopper satisfaction, highlighting Walmart's role as a one-stop shopping destination.

**Convenience**

The convenience of shopping at Walmart is another strong theme noted by many respondents. Customers value the store’s accessibility, often located nearby, and the availability of services like curbside pickup and online shopping. This theme is significant as it reflects customers’ desires for efficient shopping experiences, especially in today's fast-paced lifestyle. Enhanced convenience not only meets customer expectations but may also drive repeat visits and member loyalty.

**Helpful Staff and Customer Service**

Several customers express positive sentiments about Walmart's staff, noting their friendliness and willingness to assist shoppers. This theme is crucial as it reflects on the overall customer experience, suggesting that good customer service can noticeably enhance satisfaction. Positive interactions with employees may encourage customers to return, as they feel valued and supported during their shopping experiences.